In [83]:
import time
import json
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [84]:
def get_audio_features(genre, offset):
    client_credentials_manager = SpotifyClientCredentials(client_id="dc203630e8d74bdc9c81cae2d999ff08", client_secret="965cd714d12748d8ae2cd10aef14c9c4")
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
    playlists = sp.search(q=genre, type='playlist', limit=1)
    if playlists['playlists']['items']:
        playlist = playlists['playlists']['items'][0]
        playlist_id = playlist['id']
        playlist_tracks = sp.playlist_tracks(playlist_id, limit=100, offset=offset)
        tracks = playlist_tracks['items']
        audio_features = []
        for track in tracks:
            track_id = track['track']['id']
            audio_feature = sp.audio_features(track_id)[0]
            audio_features.append(audio_feature)
        return audio_features,tracks
        
    else:
        print(f"No playlist found for the genre '{genre}'")

In [85]:
genres = ['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'black-metal', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm',
       'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk',
       'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove',
       'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian',
       'indie-pop', 'indie', 'industrial', 'iranian', 'j-dance', 'j-idol',
       'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino',
       'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb',
       'new-age', 'opera', 'pagode', 'party', 'piano', 'pop-film', 'pop',
       'power-pop', 'progressive-house', 'psych-rock', 'punk-rock',
       'punk', 'r-n-b', 'reggae', 'reggaeton', 'rock-n-roll', 'rock',
       'rockabilly', 'romance', 'sad', 'salsa', 'samba', 'sertanejo',
       'show-tunes', 'singer-songwriter', 'ska', 'sleep', 'songwriter',
       'soul', 'spanish', 'study', 'swedish', 'synth-pop', 'tango',
       'techno', 'trance', 'trip-hop', 'turkish', 'world-music']

In [86]:
genre = 'acoustic'
track_features = []
tracks_list = []
for i in range(10):
    audio_features_list,tracks_list_temp = get_audio_features(genre,100*i)
    track_features = track_features+audio_features_list
    tracks_list = tracks_list+tracks_list_temp
    time.sleep(15)

In [87]:
for i in range(len(track_features)):
    del track_features[i]['uri']
    del track_features[i]['track_href']
    del track_features[i]['analysis_url']
    del track_features[i]['id']
    track_features[i]['track_id'] = tracks_list[i]['track']['id']
    track_features[i]['album_name'] = tracks_list[i]['track']['album']['name']
    track_features[i]['artists'] = tracks_list[i]['track']['artists'][0]['name']
    track_features[i]['popularity'] = tracks_list[i]['track']['popularity']
    track_features[i]['track_name'] = tracks_list[i]['track']['name']
    track_features[i]['explicit'] = tracks_list[i]['track']['explicit']

In [88]:
data_file = open('genre_'+genre+'.csv','w')
csv_writer = csv.writer(data_file)
count = 0
 
for track_feature in track_features:
    if count == 0:
        header = track_feature.keys()
        csv_writer.writerow(header)
        count += 1
 
    csv_writer.writerow(track_feature.values())
 
data_file.close()